<a href="https://colab.research.google.com/github/Emanuel3queijos/AWS-University-ETL/blob/main/python_scripts/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicializar bibliotecas

In [ ]:
!wget https://emanuel3queijos.github.io/AWS-University-ETL/python_scripts/requirements.txt -O requirements.txt
!sed -i '/pywin32/d' requirements.txt
!pip install -r requirements.txt


# Scripts para adicionar data dentro do banco



In [ ]:
import psycopg2

def create_connection():
    try:
        conn = psycopg2.connect(
            host="manu-rds-instance.covgxmxbd2or.us-east-1.rds.amazonaws.com",
            dbname="aws-university-etl-db",
            user="manu",
            password="Emanu0710**",
            port=5432
        )

        return conn
    except Exception as error:
        print(f"Erro ao conectar ao banco de dados: {error}")
        return None
create_connection()

In [ ]:
def inserir_departamentos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            departamentos = [
                ("Escola de Tecnologia", "Departamento focado em áreas tecnológicas."),
                ("Escola de Engenharia", "Departamento voltado para as engenharias."),
                ("Escola de Saúde", "Departamento voltado para ciências da saúde."),
                ("Ciências Biológicas", "Departamento voltado para biologia e afins.")
            ]
            for nome, descricao in departamentos:
                cur.execute("""
                    INSERT INTO tb_departamento (nome, descricao)
                    VALUES (%s, %s)
                    RETURNING id, nome
                """, (nome, descricao))
                cod_departamento = cur.fetchone()[0]
                print(f"Departamento {nome, id} inserido com sucesso. Código: {cod_departamento}")

            conn.commit()
            print("Departamentos e inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_departamentos()


In [ ]:
def inserir_cursos():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            cursos_por_departamento = {
                "Escola de Tecnologia": [
                    ("Engenharia de Software", "Curso voltado ao desenvolvimento de software, abrangendo desde a análise de requisitos até o design e implementação de sistemas.", 4),
                    ("Ciência da Computação", "Foco em computação e sistemas, incluindo programação, algoritmos, estrutura de dados e sistemas operacionais.", 4),
                    ("Inteligência Artificial", "Curso para aprendizado de IA, cobrindo machine learning, deep learning e processamento de linguagem natural.", 4),
                    ("Redes de Computadores", "Infraestrutura e comunicação de redes, abordando protocolos, segurança e administração de redes.", 3),
                    ("Segurança da Informação", "Proteção de dados e sistemas, com ênfase em criptografia, segurança cibernética e políticas de segurança.", 4),
                    ("Data Science", "Análise de dados e aprendizado de máquina, focando em big data, estatística e visualização de dados.", 4)
                ],
                "Escola de Engenharia": [
                    ("Engenharia Civil", "Foco em obras e infraestrutura, cobrindo construção, planejamento urbano e engenharia estrutural.", 5),
                    ("Engenharia Elétrica", "Curso de sistemas elétricos, abordando circuitos, eletrônica de potência e sistemas de energia.", 5),
                    ("Engenharia Mecânica", "Curso sobre mecânica aplicada, com tópicos em termodinâmica, mecânica dos sólidos e engenharia de materiais.", 5),
                    ("Engenharia de Controle e Automação", "Automação industrial, incluindo controle de processos, robótica e sistemas de automação.", 5),
                    ("Engenharia Química", "Processos químicos industriais, com foco em reações químicas, transferência de calor e projetos de plantas químicas.", 5),
                    ("Engenharia de Produção", "Gestão de processos produtivos, cobrindo otimização, logística e gerenciamento da cadeia de suprimentos.", 4)
                ],
                "Escola de Saúde": [
                    ("Medicina", "Foco em prática médica, incluindo anatomia, fisiologia, farmacologia e prática clínica.", 6),
                    ("Enfermagem", "Prática de enfermagem, abordando cuidados de saúde, administração de medicamentos e suporte ao paciente.", 4),
                    ("Farmácia", "Estudos sobre medicamentos, desde a química farmacêutica até a aplicação clínica de terapias.", 4),
                    ("Biomedicina", "Ciência biomédica, com estudos em microbiologia, imunologia e técnicas laboratoriais.", 4),
                    ("Odontologia", "Estudos odontológicos, incluindo anatomia bucal, técnicas de restauração e saúde dental.", 5),
                    ("Fisioterapia", "Reabilitação e cuidados físicos, com foco em técnicas de fisioterapia e reabilitação motora.", 4)
                ],
                "Ciências Biológicas": [
                    ("Biologia", "Estudo da vida, abrangendo ecossistemas, biodiversidade e processos biológicos.", 4),
                    ("Ecologia", "Curso sobre ecossistemas, explorando interações entre organismos e seus habitats.", 4),
                    ("Zoologia", "Estudo dos animais, abordando comportamento, anatomia e evolução de diferentes espécies.", 4),
                    ("Botânica", "Estudo das plantas, com foco em fisiologia vegetal, ecologia e identificação de espécies.", 4),
                    ("Genética", "Estudos sobre hereditariedade, incluindo genética molecular, mutações e engenharia genética.", 4),
                    ("Microbiologia", "Estudo de microrganismos, como bactérias, vírus e fungos, e suas aplicações na saúde e indústria.", 4)
                ]
            }

            for nome_departamento, cursos in cursos_por_departamento.items():
                cur.execute("""
                    SELECT id FROM tb_departamento WHERE nome = %s
                """, (nome_departamento,))
                id_departamento = cur.fetchone()

                if id_departamento is None:
                    print(f"Departamento '{nome_departamento}' não encontrado.")
                    continue

                id_departamento = id_departamento[0]

                for nome_curso, descricao_curso, duracao in cursos:
                    cur.execute("""
                        INSERT INTO tb_curso (nome, descricao, id_departamento, duracao)
                        VALUES (%s, %s, %s, %s)
                        RETURNING nome
                    """, (nome_curso, descricao_curso, id_departamento, duracao))
                    nome = cur.fetchone()[0]
                    print(f"Curso {nome_curso} inserido com sucesso.")

            conn.commit()
            print("Departamentos, cursos e disciplinas inseridos com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_cursos()


In [ ]:
from faker import Faker
import psycopg2
import random

fake = Faker('pt_BR')


def gerar_disciplinas_repetidas():
    disciplinas = []
    for _ in range(50):
        disciplina = {
            "nome": fake.sentence(nb_words=3),
            "creditos": fake.random_int(min=60, max=80),
            "departamento_id": None
        }
        disciplinas.append(disciplina)
    return disciplinas

def inserir_disciplinas():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()

            cur.execute("SELECT id, nome FROM tb_departamento")
            departamentos = cur.fetchall()

            disciplinas_base = gerar_disciplinas_repetidas()

            for id_departamento, nome_departamento in departamentos:
                print(f"Processando departamento: {nome_departamento} (ID: {id_departamento})")
                cur.execute("""
                    SELECT id, nome FROM tb_curso WHERE id_departamento = %s
                """, (id_departamento,))
                cursos = cur.fetchall()

                if cursos:
                    for curso_id, nome_curso in cursos:
                        print(f"Adicionando disciplinas ao curso: {nome_curso} (ID: {curso_id})")

                        for disciplina_base in disciplinas_base:
                            nome_disciplina = disciplina_base["nome"]
                            creditos = disciplina_base["creditos"]
                            cur.execute("""
                                INSERT INTO tb_disciplina (nome, creditos, id_departamento)
                                VALUES (%s, %s, %s)
                                RETURNING id
                            """, (nome_disciplina, creditos, id_departamento))
                            cod_disciplina = cur.fetchone()[0]

                            cur.execute("""
                                INSERT INTO tb_curso_disciplina (id_curso, id_disciplina)
                                VALUES (%s, %s)
                            """, (curso_id, cod_disciplina))

                            print(f"Disciplina '{nome_disciplina}' inserida no curso '{nome_curso}' (ID: {curso_id}) do departamento {nome_departamento}.")

                else:
                    print(f"Nenhum curso encontrado para o departamento {nome_departamento}.")

            conn.commit()
            print("Disciplinas e associações inseridas com sucesso.")

        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_disciplinas()


In [ ]:
def inserir_professores_por_disciplina():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
            conn.autocommit = False

            cur.execute("SELECT id, id_departamento FROM tb_disciplina")
            disciplinas = cur.fetchall()
            if not disciplinas:
                print("Nenhuma disciplina encontrada. Abortando.")
                return

            professores_necessarios = {}
            total_professores = 0
            for disciplina_id, _ in disciplinas:
                quantidade = random.randint(7, 15)
                professores_necessarios[disciplina_id] = quantidade
                total_professores += quantidade

            print(f"Total de professores a serem gerados: {total_professores}")

            professores = []
            professor_disciplina = set()

            matricula_inicial = 2_000_000

            for disciplina_id, quantidade in professores_necessarios.items():
                for _ in range(quantidade):
                    nome = fake.first_name()
                    sobrenome = fake.last_name()
                    nome_completo = f"{nome} {sobrenome}"
                    matricula = matricula_inicial
                    matricula_inicial += 1
                    email = f"{fake.user_name()}.{matricula}@ucsal.edu.br"
                    telefone = fake.phone_number()[:15]
                    endereco = fake.address()
                    cpf = fake.unique.numerify('###########')

                    id_departamento = next(d[1] for d in disciplinas if d[0] == disciplina_id)
                    professores.append((nome_completo, email, id_departamento, telefone, endereco, matricula, cpf))

                    semestre = random.randint(1, 9)
                    descricao = fake.sentence(nb_words=6)
                    professor_disciplina.add((matricula, disciplina_id, semestre, descricao))

            cur.executemany("""
                INSERT INTO tb_professor (nome_professor, email_professor, id_departamento, telefone, endereco, matricula, cpf)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, professores)

            cur.executemany("""
                INSERT INTO tb_professor_disciplina (matricula_professor, id_disciplina, semestre, descricao)
                VALUES (%s, %s, %s, %s)
            """, list(professor_disciplina))

            conn.commit()
            print("Professores e associações inseridos com sucesso.")
        except Exception as error:
            print(f"Erro ao inserir professores: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()

inserir_professores_por_disciplina()


In [ ]:
from faker import Faker
import random
import psycopg2

fake = Faker('pt_BR')

def calcular_desconto(media_global):
    """Calcula o desconto com base na média global do aluno."""
    if media_global >= 8:
        tipo_desconto = "Bolsa Mérito"
        valor_desconto = round(random.uniform(500, 1000), 2)
    elif 6 <= media_global < 8:
        tipo_desconto = "Bolsa Parcial"
        valor_desconto = round(random.uniform(200, 500), 2)
    else:
        tipo_desconto = "Sem Desconto"
        valor_desconto = 0.0
    return tipo_desconto, valor_desconto

def inserir_alunos_por_curso():
    """Insere todos os alunos de um curso de uma só vez."""
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
            conn.autocommit = False

            cur.execute("""
                SELECT c.id AS id_curso, c.id_departamento, c.nome AS nome_curso, COUNT(d.id) AS num_disciplinas
                FROM tb_curso c
                JOIN tb_disciplina d ON d.id_departamento = c.id_departamento
                GROUP BY c.id, c.id_departamento, c.nome
            """)
            cursos = cur.fetchall()

            if not cursos:
                print("Nenhum curso encontrado. Abortando.")
                return

            matricula_inicial = 1_000_000
            matriculas_usadas = set()
            cpfs_usados = set()

            for curso in cursos:
                id_curso, id_departamento, nome_curso, num_disciplinas = curso
                alunos_por_curso = random.randint(3000, 10000)

                print(f"Gerando {alunos_por_curso} alunos para o curso '{nome_curso}'.")

                cur.execute("SELECT id, nome FROM tb_disciplina WHERE id_departamento = %s", (id_departamento,))
                disciplinas_curso = cur.fetchall()

                alunos_dados = []
                descontos_dados = []
                disciplinas_dados = []

                for _ in range(alunos_por_curso):
                    nome_aluno = fake.unique.name()
                    matricula = matricula_inicial
                    matricula_inicial += 1
                    email = f"{fake.user_name()}.{matricula}@ucsal.edu.br"
                    cpf = fake.unique.numerify('###########')
                    media_global = round(random.uniform(0, 10), 2)
                    status = random.choice(['A', 'I'])

                    if matricula in matriculas_usadas or cpf in cpfs_usados:
                        continue

                    matriculas_usadas.add(matricula)
                    cpfs_usados.add(cpf)

                    alunos_dados.append((nome_aluno, email, cpf, media_global, status, id_curso, matricula, nome_curso))

                    tipo_desconto, valor_desconto = calcular_desconto(media_global)
                    if valor_desconto > 0:
                        descontos_dados.append((matricula, tipo_desconto, valor_desconto))

                    num_disciplinas_aluno = min(random.randint(6, 10), len(disciplinas_curso))
                    for _ in range(num_disciplinas_aluno):
                        if disciplinas_curso:
                            disciplina = random.choice(disciplinas_curso)
                            id_disciplina, nome_disciplina = disciplina
                            ano = random.randint(2015, 2023)
                            semestre = random.randint(1, 9)
                            nota_final = round(random.uniform(0, 10), 2)
                            status_disciplina = 'aprovado' if nota_final >= 5 else 'reprovado'
                            nota1 = round(random.uniform(0, 5), 2)
                            nota2 = round(random.uniform(0, 5), 2)

                            disciplinas_dados.append((matricula, id_disciplina, nome_disciplina, ano, semestre, nota_final, status_disciplina, nota1, nota2))

                if alunos_dados:
                    cur.executemany("""
                        INSERT INTO tb_aluno (nome_aluno, email, cpf, media_global, status, id_curso, matricula, nome_curso)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    """, alunos_dados)

                if descontos_dados:
                    cur.executemany("""
                        INSERT INTO tb_desconto (matricula_aluno, tipo_desconto, valor_desconto)
                        VALUES (%s, %s, %s)
                    """, descontos_dados)

                if disciplinas_dados:
                    cur.executemany("""
                        INSERT INTO tb_aluno_disciplina (matricula_aluno, id_disciplina, nome_disciplina, ano, semestre, nota_final, status_disciplina, nota1, nota2)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, disciplinas_dados)

                conn.commit()
                print(f"Dados para o curso '{nome_curso}' inseridos com sucesso.")

            print("Todos os cursos processados com sucesso.")
        except Exception as error:
            print(f"Erro ao inserir dados: {error}")
            conn.rollback()
        finally:
            cur.close()
            conn.close()


inserir_alunos_por_curso()
